# Request NDAWN Data

In [12]:
import pandas as pd
import requests
from datetime import date
from io import StringIO

class ndawn_request:

    def __init__(self, startDate='YYYY-MM-DD', endDate='YYYY-MM-DD', ontology = None, location = None, save = False):

        self.start = startDate

        self.end = endDate

        # List of ontology terms, and their URL codes to build request URL
        self.ontology = {
            'Air Temperature': ['variable=hdt', 'variable=hdt9'],
            'Relative Humidity': ['variable=hdrh', 'variable=hdrh9'],
            'Soil Temperature': ['variable=hdbst', 'variable=hdtst'],
            'Wind Speed': ['variable=hdws', 'variable=hdmxws', 'variable=hdws10', 'variable=hdmxws10'],
            'Wind Direction': ['variable=hdwd', 'variable=hdsdwd', '&variable=hdwd10', 'variable=hdsdwd10'],
            'Solar Radiation': ['variable=hdsr'],
            'Rainfall': ['variable=hdr'],
            'Air Pressure': ['variable=hdbp'],
            'Dew Point': ['variable=hddp'],
            'Wind Chill': ['variable=hdwc']
        }
        # Concatenate the ontology keys into a list for exception printout later
        ontologiesErrorMessage = '\n'.join(list(self.ontology.keys()))

        # List of stations, and URL codes to build request URL
        self.stations = {
            'Ada': 78,
            'Becker': 118,
            'Campbell': 87,
            'Clarissa': 124,
            'Eldred': 2,
            'Fox': 93,
            'Greenbush': 70,
            'Hubbard': 119,
            'Humboldt': 4,
            'Kennedy': 82,
            'Little Falls': 120,
            'Mavie': 71,
            'Ottertail': 103,
            'Parkers Prairie': 116,
            'Perham': 114,
            'Perley': 3,
            'Pine Point': 115,
            'Rice': 121,
            'Roseau': 61,
            'Sabin': 60,
            'Staples': 122,
            'Stephen': 5,
            'Ulen': 91,
            'Wadena': 117,
            'Warren': 6,
            'Waukon': 92,
            'Westport': 123,
            'Williams': 95
        }
        # Concatenate station names into a list for exception printout later
        stationsErrorMessage = '\n'.join(list(self.stations.keys()))

        self.save = save

        # This checks the start and end dates supplied to make sure they are valid
        # Start by converting dates into iso format
        startDateCheck = date.fromisoformat(startDate)
        endDateCheck = date.fromisoformat(endDate)
        # If start date is after end date, raise exception
        if startDateCheck > endDateCheck:
            raise Exception('End date cannot be before start date')
        
        # Create empty list to hold URL codes for ontology terms
        self.activeMeasures = []
        # If user supplies ontology terms
        if ontology is not None:
            for item in ontology:
                # If user-supplied term is not in the dictionary, raise exception
                if item not in self.ontology.keys():
                    raise Exception('Ontology term [' + str(item) + '] not recognized. Available ontology terms include: ' + '\n' + ontologiesErrorMessage)
                # Otherwise, append URL codes for ontology terms into the list of measurements to be requested
                else:
                    for code in self.ontology[item]:
                        self.activeMeasures.append(code)  
        # If user does not supply ontology terms, add all URL codes in dictionary to the list of measurements to be requested    
        else:
            for key in self.ontology:
                for code in self.ontology[key]:
                    self.activeMeasures.append(code)

        # Create empty list to hold URL codes for stations
        self.activeStations = []
        # If user supplies station names
        if location is not None:
            for name in location:
                # If user-supplied name is not in the dictionary, raise exception
                if name not in self.stations.keys():
                    raise Exception('Station [' + str(name) + '] not recognized. Available stations include: ' + '\n' + stationsErrorMessage)
                # Otherwise, append URL codes for stations into the list of stations to be requested
                else:
                    self.activeStations.append('station=' + str(self.stations[name]))
        # If user does not supply station names, add all station URL codes in dictionary to the list of stations to be requested
        else:    
            for key in self.stations:
                self.activeStations.append('station=' + str(self.stations[key]))

    def get_data(self):
        
        # Construct API call for the request
        baseURL = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        stations = '&'.join(self.activeStations)
        measurements = '&'.join(self.activeMeasures)
        options = '&ttype=hourly&quick_pick=&begin_date=' + self.start + '&end_date=' + self.end
        finalURL = str(baseURL + stations + '&' + measurements + options)
        
        # Request page
        page = requests.get(finalURL)
        # If status code not 200, raise exception
        if page.status_code != 200:
            raise Exception('URL request status not 200. Status code = ' + page.status_code)

        print('Request successful')

        # Convert csv data to string
        content = str(page.content)
        # Remove large, unnecessary header
        trimContent = content[content.find('Station'):len(content)]
        # Replace newline/return with string literal newline
        formatContent = trimContent.replace('\\r\\n', '\n')
        # Convert content to file object
        contentFile = StringIO(formatContent)

        # Read content into pandas dataframe. Second header row contains units
        ndawnData = pd.read_csv(contentFile, header = [0, 1])
        
        # Concatenate headers to include units
        # Assign column list to object
        columnHeaders = list(ndawnData.columns)
        # List of new headers
        newHeaderList = []
        # Iterate through column names
        for number in range(0, len(columnHeaders)):
            # If no unit, keep header unchanged, pass into new list
            if 'Unnamed' in columnHeaders[number][1]:
                newHeaderList.append(columnHeaders[number][0])
            # If unit exists, concatenate header and unit, pass into new list
            else:
                newHeader = columnHeaders[number][0] + ' (' + columnHeaders[number][1] + ') '
                newHeaderList.append(newHeader)
        # Assign new column names
        ndawnData.columns = newHeaderList

        # Create single column for datetime
        ndawnData['Date'] = pd.to_datetime(ndawnData[['Year', 'Month', 'Day']])
        
        # Save to csv if save option selected
        if self.save:
            ndawnData.to_csv('ndawnData.csv', index=False)

        return ndawnData
    

Request = ndawn_request(startDate='2021-10-24', endDate='2021-11-20', ontology=['Air Temperature'], location=['Ada','Becker','Campbell','Clarissa','Eldred','Fox','Greenbush','Hubbard','Humboldt','Kennedy','Little Falls','Mavie','Ottertail','Parkers Prairie','Perham','Perley','Pine Point','Rice','Roseau','Sabin','Staples','Stephen','Ulen','Wadena','Warren','Waukon','Westport','Williams'])
ndawnDF = Request.get_data()

Request successful


In [13]:
ndawnDF

,Station Name,Latitude (deg),Longitude (deg),Elevation (ft),Year,Month,Day,Hour (CST),Avg Air Temp (Degrees F),Avg Air Temp Flag,Avg Air Temp at 9 m (Degrees F),Avg Air Temp at 9 m Flag,Date
0,Ada,47.321100,-96.513900,910.0,2021.0,10.0,24.0,100.0,28.756,NaN,NaN,NaN,2021-10-24
1,Ada,47.321100,-96.513900,910.0,2021.0,10.0,24.0,200.0,29.043,NaN,NaN,NaN,2021-10-24
2,Ada,47.321100,-96.513900,910.0,2021.0,10.0,24.0,300.0,29.385,NaN,NaN,NaN,2021-10-24
3,Ada,47.321100,-96.513900,910.0,2021.0,10.0,24.0,400.0,31.113,NaN,NaN,NaN,2021-10-24
4,Ada,47.321100,-96.513900,910.0,2021.0,10.0,24.0,500.0,32.153,NaN,NaN,NaN,2021-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18812,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,20.0,2100.0,27.527,NaN,NaN,NaN,2021-11-20
18813,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,20.0,2200.0,27.385,NaN,NaN,NaN,2021-11-20
18814,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,20.0,2300.0,28.926,NaN,NaN,NaN,2021-11-20
18815,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,20.0,2400.0,30.034,NaN,NaN,NaN,2021-11-20


In [15]:
ndawnDF.to_csv('Temperature.csv', index=False)

# Import Data and Calculate the Sum

In [17]:
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb", workspace=r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb"):
    arcpy.defense.CoordinateTableToPoint(r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Temperature.csv", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb\Temperature", "Longitude (deg)", "DD_2", "Latitude (deg)", None)

In [19]:
arcpy.analysis.PairwiseDissolve("Temperature", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb\Mean_Temp", "Station_Name", "Station_Name FIRST;Avg_Air_Temp__Degrees_F_ MEAN", "MULTI_PART")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\lab4\\Lab4\\Lab4.gdb\\Mean_Temp'>

# Interpolation Methods

## IDW

In [21]:
arcpy.ddd.Idw("Mean_Temp", "MEAN_Avg_Air_Temp__Degrees_F_", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb\IDW_Temp", 0.0132, 1, "VARIABLE 12", None)

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\lab4\\Lab4\\Lab4.gdb\\IDW_Temp'>

## Kriging

In [22]:
arcpy.ddd.Kriging("Mean_Temp", "MEAN_Avg_Air_Temp__Degrees_F_", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb\Kriging_Temp", "Spherical # # # #", 0.0132, "VARIABLE 12", None)

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\lab4\\Lab4\\Lab4.gdb\\Kriging_Temp'>

## Spline

In [23]:
arcpy.ddd.Spline("Mean_Temp", "MEAN_Avg_Air_Temp__Degrees_F_", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\lab4\Lab4\Lab4.gdb\Spline_Temp", 0.0132, "REGULARIZED", 0.1, 12)

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\lab4\\Lab4\\Lab4.gdb\\Spline_Temp'>